In [1]:
user_data = sc.textFile("ml-100k/u.user")
user_fields = user_data.map(lambda line: line.split("|"))

In [2]:
all_occupations = user_fields.map(lambda fields: fields[3]).distinct().collect()
all_occupations.sort()

idx = 0
all_occupations_dict = {}
for o in all_occupations:
    all_occupations_dict[o] = idx
    idx += 1

print "Encoding of 'doctor': %d" % all_occupations_dict['doctor']
print "Encoding of 'programmer': %d" % all_occupations_dict['programmer']

Encoding of 'doctor': 2
Encoding of 'programmer': 14


In [3]:
import numpy as np

k = len(all_occupations_dict)
binary_x = np.zeros(k)
k_programmer = all_occupations_dict['programmer']
binary_x[k_programmer] = 1

print "Binary feature vector: %s" % binary_x
print "Length of binary vector: %d" % k

Binary feature vector: [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.
  0.  0.  0.]
Length of binary vector: 21


In [4]:
rating_data = sc.textFile("ml-100k/u.data")
rating_fields = rating_data.map(lambda line: line.split("\t"))
rating_fields.take(5)

[[u'196', u'242', u'3', u'881250949'],
 [u'186', u'302', u'3', u'891717742'],
 [u'22', u'377', u'1', u'878887116'],
 [u'244', u'51', u'2', u'880606923'],
 [u'166', u'346', u'1', u'886397596']]

In [5]:
def extract_datetime(ts):
    import datetime
    return datetime.datetime.fromtimestamp(ts)

timestamps = rating_fields.map(lambda fields: int(fields[3]))
hour_of_day = timestamps.map(lambda ts: extract_datetime(ts).hour)
hour_of_day.take(5)

[16, 21, 8, 6, 6]

In [6]:
def assign_tod(hr):
    times_of_day = {
        'morning' : range(7, 12),
        'lunch' : range(12, 14),
        'afternoon' : range(14, 18),
        'evening' : range(18, 23),
        'night' : range(23, 24) + range(0, 7),
    }    
    for k, v in times_of_day.iteritems():
        if hr in v:
            return k
        
time_of_day = hour_of_day.map(lambda hr: assign_tod(hr))
time_of_day.take(5)

['afternoon', 'evening', 'morning', 'night', 'night']

In [7]:
np.random.seed(42)

x = np.random.randn(10) # 返回常態分佈樣本
norm_of_x = np.linalg.norm(x) # 計算二階泛數
normalized_x = x / norm_of_x # 正則化特徵向量
norm_of_normalized_x = np.linalg.norm(normalized_x) # 計算正則化向量的二階泛數

print x
print norm_of_x
print normalized_x
print norm_of_normalized_x

[ 0.49671415 -0.1382643   0.64768854  1.52302986 -0.23415337 -0.23413696
  1.57921282  0.76743473 -0.46947439  0.54256004]
2.59080239507
[ 0.19172213 -0.05336737  0.24999534  0.58786029 -0.09037871 -0.09037237
  0.60954584  0.29621508 -0.1812081   0.20941776]
1.0


In [8]:
norm_of_x_by_hand = np.sqrt(reduce(lambda a,b: a+b, map(lambda n: n*n, x)))
print norm_of_x_by_hand

2.59080239507


In [9]:
from pyspark.mllib.feature import Normalizer

normalizer = Normalizer()
vector = sc.parallelize([x])

normalized_x = normalizer.transform(vector).first().toArray()
print normalized_x

[ 0.19172213 -0.05336737  0.24999534  0.58786029 -0.09037871 -0.09037237
  0.60954584  0.29621508 -0.1812081   0.20941776]
